# Домашняя работа 2
## Часть 1: Байесовская классификация

**Задание 1** 
*Открыть в Pandas файл `names.csv`. Ответить на вопросы ниже, используя средства языка Python и необходимых библиотек*

In [2]:
import pandas as pd

names_data = pd.read_csv('baby-names.csv')

print(names_data.shape)
names_data.head()

(258000, 4)


,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy


**Задание 2** 
*Разделить данные в выборке на обучающий набор и тестирование (выбор принципа разделения за вами – например, 70% данных в обучении
и 30% в тестировании).*

In [3]:
from sklearn.model_selection import train_test_split

names_data_reduced = names_data[['name', 'sex']]

train_data, test_data = train_test_split(names_data_reduced,
                                         test_size=0.3,
                                         random_state=42,
                                         stratify=names_data_reduced['sex'])

print(f'Размер тренировочной выборки: {train_data.shape}.\n'
      f'Размер тестовой выборки: {test_data.shape}.')

train_data.head()

Размер тренировочной выборки: (180600, 2).
Размер тестовой выборки: (77400, 2).


,name,sex
82546,Thaddeus,boy
118648,Shannon,boy
15196,Alva,boy
34177,Lyle,boy
17628,June,boy


**Задание 3** 
*Обучить наивную байесовскую классификацию из файла `Sem2.ipynb` (см. вложения) на тренировочном наборе данных. Затем с помощью
метода `classify()` разметить имена по полу в тестировочном наборе данных.*

In [4]:
from collections import defaultdict
from math import log


def train(samples):
    classes, freq = defaultdict(lambda: 0), defaultdict(lambda: 0)
    for feats, label in samples:
        classes[label] += 1
        for feat in feats:
            freq[label, feat] += 1

    for label, feat in freq:
        freq[label, feat] /= classes[label]
    for c in classes:
        classes[c] /= len(samples)
    return classes, freq


def classify(classifier_, feats):
    classes, prob = classifier_
    return min(classes.keys(),
               key=lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl, feat), 10 ** (-7)))
                                                      for feat in feats))


def get_features(sample):
    return sample[-1]


train_samples = [(get_features(name), sex)
                 for name, sex in zip(train_data['name'],
                                      train_data['sex'])]
classifier = train(train_samples)
classifier

(defaultdict(<function __main__.train.<locals>.<lambda>()>,
             {'boy': 0.5, 'girl': 0.5}),
 defaultdict(<function __main__.train.<locals>.<lambda>()>,
             {('boy', 's'): 0.07830564784053157,
              ('boy', 'n'): 0.2145514950166113,
              ('boy', 'a'): 0.016345514950166114,
              ('boy', 'e'): 0.13782945736434107,
              ('boy', 'm'): 0.016334440753045402,
              ('girl', 'a'): 0.38447397563676633,
              ('girl', 'h'): 0.024573643410852712,
              ('boy', 'o'): 0.0640531561461794,
              ('girl', 'e'): 0.3011295681063123,
              ('boy', 'd'): 0.07270210409745294,
              ('boy', 'r'): 0.060398671096345516,
              ('boy', 'c'): 0.009158361018826135,
              ('boy', 't'): 0.05179401993355482,
              ('boy', 'l'): 0.0825359911406423,
              ('girl', 'r'): 0.011638981173864894,
              ('girl', 'n'): 0.08035437430786269,
              ('girl', 'y'): 0.07866002214839424

**Задание 4** 
*Посчитайте среднюю долю правильных "ответов"классификатора. Какие еще метрики можно построить, чтобы оценить, насколько хорошо справился с задачей данный классификатор?*

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

test_samples = [(get_features(name), sex)
                for name, sex in zip(test_data['name'],
                                     test_data['sex'])]

predicted = [classify(classifier, feat) for feat, _ in test_samples]
actual = [sex for _, sex in test_samples]

accuracy = accuracy_score(actual, predicted)

precision = precision_score(actual, predicted, pos_label='boy')
recall = recall_score(actual, predicted, pos_label='boy')
f1 = f1_score(actual, predicted, pos_label='boy')
conf_matrix = confusion_matrix(actual, predicted, labels=['girl', 'boy'])

print(f'Accuracy {accuracy:.3f}')
print(f'Precision: {precision:.3f} \n'
      f'Recall: {recall:.3f}\n'
      f'F1-score: {f1:.3f}')
print(f'Матрица ошибок:\n{conf_matrix}')

Accuracy 0.778
Precision: 0.748 
Recall: 0.839
F1-score: 0.791
Матрица ошибок:
[[27740 10960]
 [ 6218 32482]]


*Классификатор справился довольно хорошо, учитывая, что был использован всего один признак (последняя буква имени), показав точность 77.8%. Также я использовал:*
1. *`Precision`(определяет, как много из объектов, которые классификатор отнес к положительному классу, действительно являются положительными)*
2. *`Recall`(это доля истинно положительных среди всех реальных положительных)*
3. *`F1-score`(гармоническое среднее между точностью и полнотой, которое стремится сбалансировать эти две метрики)*
4. *`Confusion Matrix`(показывает, сколько образцов каждого класса было классифицировано верно и сколько было ошибочно классифицировано).*

**Задание 5** 
*Модифицируйте функцию get_features() таким образом, чтобы в качестве целевого признака бралась другая структура (не последняя буква имени). Возможно, это будет набор из первой и последней буквы. Или, например, имя целиком...*


In [7]:
def get_features_modified(sample):
    return sample[0], sample[-1]


train_samples_modified = [(get_features_modified(name), sex) for name, sex in
                          zip(train_data['name'], train_data['sex'])]

classifier_modified = train(train_samples_modified)

test_samples_modified = [(get_features_modified(name), sex) for name, sex in
                         zip(test_data['name'], test_data['sex'])]

predicted_modified = [classify(classifier_modified, feat) for feat, _ in test_samples_modified]
actual_modified = [sex for _, sex in test_samples_modified]

accuracy_modified = accuracy_score(actual_modified, predicted_modified)
print(f'Accuracy_modified: {accuracy_modified:.3f}')
# classifier_modified

Accuracy_modified: 0.781


**Задание 6** 
*Модифицируйте метод `classify()` так, чтобы вместо логарифмов брались исходные значения вероятностей, а вместо `argmin(...)` считался функционал `argmax(...)`. Также можете использовать другой метод классификации (из лекций или учебников, или модифицировать его самому методом проб и ошибок).*

In [8]:
def classify_modified(classifier_, feats):
    classes, prob = classifier_
    return max(classes.keys(),
               key=lambda cl: classes[cl] * prod(prob.get((cl, feat), 10 ** (-7)) for feat in feats))


# Вспомогательная функция для вычисления произведения элементов итерируемого объекта
def prod(iterable):
    result = 1
    for el in iterable:
        result *= el
    return result


predicted_modified_2 = [classify_modified(classifier_modified, feat) for feat, _ in test_samples_modified]
accuracy_modified_2 = accuracy_score(actual_modified, predicted_modified_2)
print(f'Accuracy_modified_2: {accuracy_modified_2:.3f}')
print(f'Разность: {(accuracy_modified_2 - accuracy):.3f}%')

Accuracy_modified_2: 0.781
Разность: 0.003%


**Задание 7** 
*Улучшилась ли доля правильных ответов алгоритма после модификации целевого признака и метода `classify()`? Какие выводы можно
сделать о выборе целевых признаков и о влиянии классифицирующей функции на результат алгоритма?*
**Ответ**
*Да, доля правильных ответов улучшилась на $0.003\%$ после модификации целевого признака, это подтверждает, что выбор подходящих признаков крайне важен, можно попробовать подобрать ещё более точные признаки. Но модификация функции `classify()` не изменила результат, это показывает, что не все изменения методов классификации улучшают результаты.* 



**Задание 8** 
*Запустите гауссовский и мультиномиальный классификатор методами из `sklearn.naive_bayes`. Насколько точна классификация в данном случае? Какой из трех методов оказался точнее (наивный, гауссовский или мультиномиальный)?*

In [9]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Преобразование категориальных признаков в числовые
encoder = LabelEncoder()

# Используем обе буквы (первую и последнюю) как признаки
X_train = [f"{name[0]}{name[-1]}" for name in train_data['name']]
X_test = [f"{name[0]}{name[-1]}" for name in test_data['name']]

# Используем CountVectorizer для преобразования букв в числовой формат, подходящий для MultinomialNB
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 1))
X_train_multinom = vectorizer.fit_transform(X_train).toarray()
X_test_multinom = vectorizer.transform(X_test).toarray()

# Кодируем метки классов
y_train = encoder.fit_transform(train_data['sex'])
y_test = encoder.transform(test_data['sex'])

# Обучение классификаторов
gnb = GaussianNB().fit(X_train_multinom, y_train)
mnb = MultinomialNB().fit(X_train_multinom, y_train)

# Оценка точности
accuracy_gnb = gnb.score(X_test_multinom, y_test)
accuracy_mnb = mnb.score(X_test_multinom, y_test)

print(f'Доля правильных ответов с помощью Гауссовского классификатора: {accuracy_gnb:.3f}\n'
      f'Доля правильных ответов с помощью мультиноминального классификатора: {accuracy_mnb:.3f}')

Доля правильных ответов с помощью Гауссовского классификатора: 0.687
Доля правильных ответов с помощью мультиноминального классификатора: 0.731


*Данные классификации не очень точно. Если сравнить результаты с Байесовским классификатором (Доля правильных ответов $= 78.1\%$), можно сделать вывод, что наша реализация самая точная, потом идёт Мультиноминальная($73.1\%$), а после Гауссовская($68.7\%$).*

## Часть 2: Классификация Ирисов

*Теперь возьмем датасет, содержащий описание цветков ириса и их классификацию по сортам (Setosa, Versicolour, Virginica). Этот набор данных содержится в `sklearn.datasets.load_iris()`.*

**Задача 1**
*Разделите данные на обучение и тестировку (аналогично заданию 1).*

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

iris = load_iris()
X, y = iris.data, iris.target

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Проверка размеров получившихся выборок
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((105, 4), (45, 4), (105,), (45,))

**Задача 2**
*С помощью метода `LDA` (линейный дискриминантный анализ) реализуйте классификацию сортов ириса на основании признаков датасета. По метрикам из задания 1 оцените эффективность классификатора*

In [13]:
# Инициализация и обучение LDA классификатора
lda = LinearDiscriminantAnalysis().fit(X_train, y_train)
# Прогнозируем значения тестовой выборки
y_pred = lda.predict(X_test)

# Оценка эффективности классификатора
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Матрица ошибок:\n {conf_matrix}')
print(f'Доля правильных ответов: {accuracy}')
print(class_report)

Матрица ошибок:
 [[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
Доля правильных ответов: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


*Результаты показывают, что классификатор LDA из sklearn сработал идеально, не допуская ошибок на тестовых данных, все параметры равны $100\%$*

**Задача 3**

*Сравните метод `LDA` из `sklearn.discriminant_analysis` и реализацию из `Sem3.ipynb` (см. вложения)*   

In [18]:
def lda_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T, X_classify[i] - mju_classify[i])

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape((len(mju), 1)),
                                          (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    # Преобразуйте собственные векторы в вещественные
    eig_vecs = eig_vecs.real

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs


# Применяем функцию LDA_dimensionality для понижения размерности
W = lda_dimensionality(X_train, y_train, 2)
X_train_lda = np.dot(X_train, W)
X_test_lda = np.dot(X_test, W)

# Применяем LDA из sklearn для классификации на преобразованных данных
lda_transformed = LinearDiscriminantAnalysis()
lda_transformed.fit(X_train_lda, y_train)
y_pred_lda_transformed = lda_transformed.predict(X_test_lda)

# Оцениваем модель на преобразованных данных
accuracy_transformed = accuracy_score(y_test, y_pred_lda_transformed)
precision_transformed = precision_score(y_test, y_pred_lda_transformed, average='weighted')
recall_transformed = recall_score(y_test, y_pred_lda_transformed, average='weighted')
f1_transformed = f1_score(y_test, y_pred_lda_transformed, average='weighted')
conf_matrix_transformed = confusion_matrix(y_test, y_pred_lda_transformed)

accuracy_transformed, precision_transformed, recall_transformed, f1_transformed, conf_matrix_transformed


(1.0,
 1.0,
 1.0,
 1.0,
 array([[19,  0,  0],
        [ 0, 13,  0],
        [ 0,  0, 13]], dtype=int64))

In [19]:
# Изменяем параметры LDA и оцениваем производительность модели
lda_shrinkage = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
lda_shrinkage.fit(X_train, y_train)
y_pred_shrinkage = lda_shrinkage.predict(X_test)

# Оценка модели
accuracy_shrinkage = accuracy_score(y_test, y_pred_shrinkage)
precision_shrinkage = precision_score(y_test, y_pred_shrinkage, average='weighted')
recall_shrinkage = recall_score(y_test, y_pred_shrinkage, average='weighted')
f1_shrinkage = f1_score(y_test, y_pred_shrinkage, average='weighted')
conf_matrix_shrinkage = confusion_matrix(y_test, y_pred_shrinkage)

accuracy_shrinkage, precision_shrinkage, recall_shrinkage, f1_shrinkage, conf_matrix_shrinkage


(1.0,
 1.0,
 1.0,
 1.0,
 array([[19,  0,  0],
        [ 0, 13,  0],
        [ 0,  0, 13]], dtype=int64))

1. Сравнение метода LDA из sklearn.discriminant_analysis и реализации функции LDA_dimensionality:

   - Метод LDA из `sklearn.discriminant_analysis` представляет готовую реализацию LDA, которая автоматически обрабатывает классы, рассчитывает собственные значения и вектора, также применяет проекцию данных. Это более удобный способ использования LDA, особенно для задач классификации. Думаю, что данный метод на больших данных работает быстрее из-за оптимизаций.
   
   - Реализация `LDA_dimensionality`, представляет собой более низкоуровневую реализацию LDA, в которой вручную вычисляются собственные значения и векторы, а затем они применяются для проекции данных. Это предоставляет большую гибкость и контроль, но также требует больше усилий для реализации и может быть менее эффективным.

2. Рассмотрение параметров метода LDA:

   Метод LDA из `sklearn.discriminant_analysis` имеет несколько параметров, которые могут сильно влиять на его эффективность:

   - `solver`: Этот параметр определяет метод, используемый для вычисления собственных значений и векторов. В зависимости от данных и размерности пространства признаков, разные методы могут быть более или менее эффективными. Например, `solver='eigen'` использует вычислительно затратный метод вычисления всех собственных значений, в то время как `solver='lsqr'` использует метод наименьших квадратов. Выбор правильного метода может значительно повлиять на производительность.

   - `shrinkage`: Этот параметр определяет метод регуляризации для случая, когда классы могут быть линейно зависимыми или когда у вас мало обучающих примеров. Это также может сильно влиять на производительность и стабильность метода.

   - `n_components`: Этот параметр позволяет установить количество компонент, которые вы хотите извлечь из LDA. Выбор правильного количества компонент также может повлиять на конечный результат.



(Я немного изменил семинарскую функцию, так как она не была ошибка ComplexWarning)
